In [2]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime

import sys
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

from quotaclimat.utils.plotly_theme import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data

In [5]:
os.listdir("../../data/cop27")

['0511', '0611', '0711', '0811', '0911', '20221108_20221112_Programme_TV.csv']

In [8]:
path = "../../data/cop27/20221108_20221112_Programme_TV.csv"

In [14]:
data = pd.read_csv(path).assign(count = lambda x : 1)
data.head()

,start,stop,day,length_minutes,channel_name,title,subtitle,date,desc_text,category_text,count
0,2022-11-08 00:05:00+01:00,2022-11-08 00:55:00+01:00,2022-11-08 00:00:00+01:00,50,TF1,New York : crime organisé,Une opération à risques,2021.0,Saison:1 - Episode:7 - La famille de Bell rend...,série policière,1
1,2022-11-08 00:55:00+01:00,2022-11-08 01:40:00+01:00,2022-11-08 00:00:00+01:00,45,TF1,New York Unité Spéciale,Un moment de faiblesse,2021.0,Saison:22 - Episode:12 - Olivia entre dans un ...,série policière,1
2,2022-11-08 01:40:00+01:00,2022-11-08 02:35:00+01:00,2022-11-08 00:00:00+01:00,55,TF1,New York : crime organisé,La mort en ligne,2021.0,Saison:1 - Episode:4 - Alors que la famille Wh...,série policière,1
3,2022-11-08 02:35:00+01:00,2022-11-08 06:25:00+01:00,2022-11-08 00:00:00+01:00,230,TF1,Programmes de la nuit,NaN,NaN,Retrouvez tous vos programmes de nuit.,programme indéterminé,1
4,2022-11-08 06:25:00+01:00,2022-11-08 08:25:00+01:00,2022-11-08 00:00:00+01:00,120,TF1,TFou,NaN,NaN,MolangBarbapapa en Famille (ST)Thomas et ses a...,magazine jeunesse,1


## Exploration

In [25]:
px.bar(
    data.groupby("category_text",as_index = False)["count"].sum().sort_values("count",ascending = False).head(30),
    x = "category_text",
    y = "count",
).show()

px.bar(
    data.groupby("category_text",as_index = False)["length_minutes"].sum().sort_values("length_minutes",ascending = False).head(30),
    x = "category_text",
    y = "length_minutes",
).show()

# Analyse des thématiques

In [26]:
from quotaclimat.data_processing.keyword_tool import KeywordsTool

kwt = KeywordsTool(case_sensitive=False,lowercase = True)

kwt.load_from_airtable(airtable_table_name="COP27",keyword_col = "name",variants_col=["alternatives_mediatree","alternatives"])

In [28]:
kwt.extract_keywords("Découvrez les prévisions météorologiques du lendemain présentées par l'équipe de CNews qui suit pour vous les dernières informations de Météo France")

['Pays']

In [31]:
data["keywords"] = data["desc_text"].map(lambda x : kwt.extract_keywords(x) if not pd.isnull(x) else [])

In [37]:
set(["coucou","hello"]).intersection(set(["coucoue"]))

set()

In [62]:
(data.join(
    data["keywords"].explode()
    .reset_index()
    .assign(count = lambda x : 1).groupby(["index","keywords"])["count"].sum()
    .unstack("keywords").fillna(0.0))
.fillna(0.0)
).to_excel("progs.xlsx")

In [63]:
! start .

In [46]:
def test_keywords(x,y):
    return len(set(x).intersection(set(y))) > 0

data["is_COP"] = data["keywords"].map(lambda x : test_keywords(x,["COP26","COP27"]))
data["is_écologie"] = data["keywords"].map(lambda x : test_keywords(x,["Climat","Ecologie","Transition énergétique","Energies fossiles","COP26","GIEC"]))

In [47]:
data.to_excel("test.xlsx")

In [40]:
! start .

In [43]:
kwt.show_keywords("""Saison:3 - Episode:45 - La jungle n'est pas toujours un endroit paisible. Les animaux qui la peuplent en savent quelque chose. Maurice, un manchot tigré, Batricia, une chauve-souris, Miguel, un gorille ou encore Gilbert, un tarsier, sont là pour assurer la sécurité de leurs amis. Pour les appeler, il suffit de tirer un signal d'alarme qui envoie de petits nuages de fumée au sommet d'un volcan.""")

In [48]:
data["keywords"].explode().dropna().value_counts()

Pays                       462
Emissions                  185
Engagement                 122
Pertes & dommages           91
Développement durable       83
Argent                      75
Ecologie                    71
Entreprises                 58
Scientifique                54
Santé                       37
Adaptation                  27
Figures politiques          23
Climat                      21
Transition énergétique      19
COPs                        16
Agriculture                 16
Accord                      15
Pêche                       12
Energies fossiles           10
Justice climatique           9
Bilan de la COP              9
Incendies                    8
Inondations                  8
Déforestation                8
Activistes                   7
Nucléaire                    6
Atténuation                  6
Electricité                  5
COP26                        3
Biodiversité                 3
Pollution                    3
Croissance                   3
Hausse d

In [ ]:
counts_total = kwt.count_keywords_on_corpus(data["text"],as_melted = True)
counts_total = counts_total.query("name!='Pays'")